In [2]:
import pandas as pd

In [8]:
import pandas as pd
import numpy as np

def gdp_to_wide(path, outfile):
    df = pd.read_excel(path)

    temp = df["Unnamed: 0"].str.split("-", expand=True)
    df["Year"] = temp[0].astype(int)
    df["Month"] = temp[1].str.strip()

    df["GDP"] = df["Monthly Real GDP Index"]

    month_map = {
        "Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,
        "Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12
    }
    df["MonthNum"] = df["Month"].map(month_map)

    df["Date"] = pd.to_datetime(dict(
        year=df["Year"], 
        month=df["MonthNum"], 
        day=1
    ))

    df = df.sort_values("Date").reset_index(drop=True)

    df["Growth"] = df["GDP"].pct_change() * 100
    df.loc[0, "Growth"] = np.nan

    wide = df.pivot(index="Year", columns="Month", values="Growth")

    ordered_months = ["Jan","Feb","Mar","Apr","May","Jun",
                      "Jul","Aug","Sep","Oct","Nov","Dec"]
    wide = wide[ordered_months]

    wide = wide.reset_index()

    wide.to_csv(outfile, index=False)

    return wide

gdp_wide = gdp_to_wide(
    "gdp_monthly_data.xlsx",
    "gdp_growth_wide.csv"
)

gdp_wide.head()


Month,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1992,NaN,0.467508,1.045959,-0.080365,-0.278450,1.497064,0.122013,-0.256694,0.412087,0.538811,0.230542,0.473471
1,1993,-0.388872,0.291372,-0.086514,0.127371,0.512512,0.218073,-0.361784,0.407088,0.756541,-0.066139,1.044562,0.259033
2,1994,-0.103194,0.574631,0.507751,0.142111,0.995586,0.043877,-0.118251,0.726943,-0.419215,0.924998,-0.107531,0.993853
3,1995,-0.033728,-0.784315,0.876990,-0.576606,0.323535,1.030894,-0.576541,0.600420,0.709252,-0.310598,0.045608,0.871886
4,1996,-0.162269,0.116779,0.722272,1.237015,-0.372391,0.467959,0.488983,0.092339,0.468149,0.817678,-0.043736,-0.281502
